In [133]:
import pandas as pd 
import numpy as np
import requests
import time
import math
import json
pd.set_option('display.max_columns', None)

**Import List of Tetra League Players**

In [3]:
from json.decoder import JSONDecodeError
#Data Collected 05/07/2023
try:
    response = requests.get("https://ch.tetr.io/api/users/lists/league/all")
    response.raise_for_status()
    data = response.json()
except JSONDecodeError:
    print("Error: Response is not in JSON format")
except requests.exceptions.HTTPError as http_err:
    print(f"HTTP error occurred: {http_err}")
except Exception as err:
    print(f"Other error occurred: {err}")

In [35]:
df = pd.json_normalize(pd.DataFrame(response.json()['data']).T,sep=',')

In [53]:
df = pd.DataFrame(data)
df1 = pd.json_normalize(df['data']).T
df2 = pd.json_normalize(df1[0],sep =',')
df2.head()

,_id,username,role,xp,supporter,verified,country,league.gamesplayed,league.gameswon,league.rating,league.glicko,league.rd,league.rank,league.bestrank,league.apm,league.pps,league.vs,league.decaying
0,615fb20fe17beeef45104302,blaarg,user,5311720.0,True,True,US,1715,1482,24999.136276,4217.154648,92.109489,x,x,169.68,3.40,327.62,True
1,5e844b0868270e617d52c990,czsmall0402,user,15653741.5,True,True,None,423,398,24999.016611,4191.582595,99.482739,x,x,180.16,3.09,348.69,False
2,5e88d0ead351fa71316ba29e,promooooooo,user,9752789.0,True,True,AQ,3702,2336,24999.006568,4162.234189,72.794460,x,x,196.83,4.25,372.41,False
3,5f5dbcc4454e942b4fdfc5fa,vincehd,user,12772255.0,True,True,PH,1183,803,24998.379175,4033.461712,72.602636,x,x,179.74,3.49,350.63,False
4,5e47696db7c60f23a497ee6c,caboozled_pie,user,17381417.5,True,True,US,6525,4052,24998.253616,4008.281969,65.305367,x,x,178.05,3.58,358.89,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
55688,642b437a6cdce157f6256d05,ziruiziyi,user,59843.0,False,False,US,76,8,146.604416,154.091398,66.366139,d,c-,3.55,0.51,7.19,False
55689,63e889ffef3dee8edfdccd6c,usanumba1,user,372026.0,False,False,US,264,90,138.920272,130.227912,88.354131,d,b,6.98,1.13,12.07,True
55690,629a13c8dea07bce3634a2de,ricafton,user,449454.0,False,False,US,118,28,137.941093,138.480098,65.227023,d,d,4.23,0.59,7.87,False
55691,632a10f36da7330727f1e07e,osheets15,user,405834.0,False,False,US,66,9,134.755387,127.260140,91.011610,d,d,2.05,0.75,3.56,True


In [47]:
df2['league.rank'].unique()

array(['x', 'u', 'ss', 's+', 's', 's-', 'a+', 'a', 'a-', 'b+', 'b', 'b-',
       'c+', 'c', 'c-', 'd+', 'z', 'd'], dtype=object)

In [54]:
df3 = df2[~(df2['league.rank']=='z')]

In [56]:
df3.shape[0]

55692

**Get a sample from the leaderboard to use as the training data for the model** 

In [57]:
stratify_column = '_id'

groupby_column = 'league.rank'

# Sample size calculated using https://www.qualtrics.com/blog/calculating-sample-size/
#Population Size = 46531, Confidence Level = 99%, Margin of Error = 5%
sample_size = 656

rank_count = len(df3['league.rank'].unique())

sample_split = math.ceil(sample_size/rank_count)

groups = df3.groupby(groupby_column)

def stratified_sample(group):
    return group.sample(min(len(group), sample_split))

sampled_df = groups.apply(stratified_sample)

sampled_df = sampled_df.reset_index(drop=True)

In [58]:
sampled_df.sort_values(by='league.rating',ascending=False).head()

,_id,username,role,xp,supporter,verified,country,league.gamesplayed,league.gameswon,league.rating,league.glicko,league.rd,league.rank,league.bestrank,league.apm,league.pps,league.vs,league.decaying
662,5e3f6ebe3dacc16dbc4dd6aa,doremy,user,12150561.0,True,True,FR,4987,3276,24992.961459,3649.497830,75.279147,x,x,143.10,3.08,305.29,False
652,5fb8a5b19efe6021cc69689d,simpshrimp,user,14114000.5,False,False,CL,1191,730,24988.905465,3529.837840,75.327832,x,x,144.70,3.31,286.42,False
659,5ef74f58efada759d102d332,yida,user,12502832.0,True,False,TW,753,531,24985.088247,3446.095934,65.495166,x,x,121.69,2.99,258.58,False
634,6030455395d4a33b1b6831bc,master101,user,12473597.5,True,True,US,4706,2643,24982.307831,3403.745212,69.884592,x,x,135.53,2.97,266.35,False
661,6017285fddd442140aca7d10,olivegarden,user,10322523.0,True,False,IT,1771,995,24975.220188,3313.476287,66.664252,x,x,122.97,2.83,257.51,False


In [60]:
sampled_df['league.rank'].value_counts()

league.rank
a     39
d     39
u     39
ss    39
s-    39
s+    39
s     39
d+    39
c-    39
a+    39
c+    39
c     39
b-    39
b+    39
b     39
a-    39
x     39
Name: count, dtype: int64

**Function to grab each of the sample's sprint data** 

In [185]:
def get_json(user_id,rank,responses):
    url = f"https://ch.tetr.io/api/users/{user_id}/records"
    time.sleep(1)
    response = requests.get(url)
    if response.status_code == 200:
        if not pd.DataFrame(response.json()['data']['records']['40l']['record']).empty:
            sprint_data = response.json()['data']['records']['40l']['record']['endcontext']
            sprint_data['replayid'] = response.json()['data']['records']['40l']['record']['replayid']
            responses.append(sprint_data)
            print(f"successfully appended: {user_id}")
            return responses
        else:
            print(f"failed to append: {user_id}")
            new_sample = df_without_sample[df_without_sample['league.rank']== rank]['_id'].sample().iloc[0]
            print(f"new sample:{new_sample}")
            while new_sample in users:
                    new_sample = df_without_sample[df_without_sample['league.rank']== rank]['_id'].sample().iloc[0]
                    print(f"new sample repeat:{new_sample}")
            df_without_sample[~(df_without_sample['_id'] == new_sample)]
            
            return get_json(new_sample,rank,responses)

In [186]:
responses = []
users = sampled_df[['_id','league.rank']]
df_without_sample = df3[~df3['_id'].isin(users['_id'])]
count = 1
for index, row in sampled_df.iterrows():
    responses = get_json(row._id,row['league.rank'],responses)
    print(f"total appended:{count}")
    count += 1


df_sprint = pd.json_normalize(responses)

successfully appended: 641ec7d248adfa7a3e7d11cd
total appended:1
failed to append: 642ee05385f1735820b01543
new sample:64099e77aa4882e7599be0f0
successfully appended: 64099e77aa4882e7599be0f0
total appended:2
successfully appended: 63c5ab12b1efb7cfd1d0049b
total appended:3
successfully appended: 6448f6211549b6a2d55749bf
total appended:4
successfully appended: 63114da0269fd63b160f4d8c
total appended:5
successfully appended: 63b84be60ee686097365869a
total appended:6
successfully appended: 62de792150f1522bfb7fe9ae
total appended:7
successfully appended: 61284ffeea073c5a9ffeff12
total appended:8
failed to append: 63f21ae040fed39b1edf3b44
new sample:636d17d06860b6f7bc3460b0
successfully appended: 636d17d06860b6f7bc3460b0
total appended:9
successfully appended: 641dcc37d16b147a445fa370
total appended:10
successfully appended: 6323df8673264300c002dc03
total appended:11
successfully appended: 5eeff29c26ae776c195c42f5
total appended:12
successfully appended: 639b36c0f1f2a5e084bb1e33
total appen

successfully appended: 61a593782d88c8b316736bc5
total appended:111
failed to append: 644a4a818b18b8aa990330a5
new sample:63b7ba420ee6860973654eab
successfully appended: 63b7ba420ee6860973654eab
total appended:112
successfully appended: 636d2f7fd49473f7b4cb53ba
total appended:113
successfully appended: 61f1f12e6a06aa4090c6b0e9
total appended:114
failed to append: 63d37c1096fc8b6d0439db98
new sample:60a2c48cb4f0c31a9f8a7ded
successfully appended: 60a2c48cb4f0c31a9f8a7ded
total appended:115
successfully appended: 61ef4388ed192d40892dc7f4
total appended:116
successfully appended: 624d8b5a0833cfa3b159942b
total appended:117
failed to append: 6447ae4344b3865c688b41fd
new sample:608128220d05676ed5d70304
successfully appended: 608128220d05676ed5d70304
total appended:118
failed to append: 64433d3444b3865c688944be
new sample:64394938eca7a176267c62f9
successfully appended: 64394938eca7a176267c62f9
total appended:119
failed to append: 6406a4b5ad41309b2b072cad
new sample:63e18b0e63226d6d117479fb
su

failed to append: 6103665125a1df32e19743db
new sample:614568bc3d6fdb44cec8e98b
failed to append: 614568bc3d6fdb44cec8e98b
new sample:6363ea63a08e507f97ba0a3b
successfully appended: 6363ea63a08e507f97ba0a3b
total appended:209
failed to append: 612bc6174f77f944f123765a
new sample:63e454b196fc8b6d043faedc
failed to append: 63e454b196fc8b6d043faedc
new sample:61a1d28f12d3f9b32b0263e0
successfully appended: 61a1d28f12d3f9b32b0263e0
total appended:210
successfully appended: 6447d2fac320444993e73022
total appended:211
successfully appended: 6174518a0e6ac854a401f784
total appended:212
successfully appended: 6425ce511599c706a7974b52
total appended:213
successfully appended: 63dcfe6596fc8b6d043d2399
total appended:214
successfully appended: 6447e94235154fed9e73c007
total appended:215
failed to append: 632a2ef329c3f40735338790
new sample:63c9665c9343cecfda0c2849
successfully appended: 63c9665c9343cecfda0c2849
total appended:216
successfully appended: 63eb2d6440fed39b1edccae6
total appended:217
su

failed to append: 6448a803c320444993e7991e
new sample:641376fd40fed39b1eeba11f
failed to append: 641376fd40fed39b1eeba11f
new sample:62a57c05568d366712e3e6cc
successfully appended: 62a57c05568d366712e3e6cc
total appended:309
successfully appended: 6142d59e3d6fdb44cec80b19
total appended:310
successfully appended: 64011de581abac9b335ee109
total appended:311
successfully appended: 643be2269e17f7a3b033dde5
total appended:312
successfully appended: 61a2bcd8dbc55fb324c299e4
total appended:313
successfully appended: 63ee6d7eec2f099b3a4e6f2a
total appended:314
successfully appended: 640a318eec2f099b3a5897ee
total appended:315
successfully appended: 63c8760b8b6199cff4f10383
total appended:316
successfully appended: 64065ed481abac9b3360c452
total appended:317
successfully appended: 6423eb7b26e2c90692ad9a6c
total appended:318
successfully appended: 63d00b5969ace36d329c9d3c
total appended:319
successfully appended: 62fbb670cc59143b0f6949a1
total appended:320
successfully appended: 644ec5690a68d94

successfully appended: 63e1468956c1df6d14d7eae4
total appended:402
successfully appended: 641ce7ec72059e7a4e38ed9a
total appended:403
successfully appended: 627a565c57421706958fea28
total appended:404
successfully appended: 6356df5d9b4a3ccfee906aa8
total appended:405
successfully appended: 640dcaabec2f099b3a59ed62
total appended:406
successfully appended: 63d154c796fc8b6d043919ab
total appended:407
successfully appended: 6443ad4056877088c315c723
total appended:408
successfully appended: 641c3e368bfb077a58ca75ce
total appended:409
successfully appended: 6149eaf63d6fdb44ceca57d5
total appended:410
successfully appended: 622348663ca16469923fcaa4
total appended:411
successfully appended: 63e1852de534b56d1ecab7fe
total appended:412
successfully appended: 643785b22f10fad0b93a2bd2
total appended:413
successfully appended: 639a5021b2a657e0981d1ae7
total appended:414
successfully appended: 61d5055b2e15a1740d5b3b26
total appended:415
successfully appended: 63e39f31e534b56d1ecb78aa
total appended

successfully appended: 62dac9190a12d27d67f193da
total appended:512
successfully appended: 63b3332d45b76b098cdaf5d8
total appended:513
successfully appended: 6373bbbe455dd1313ff431c6
total appended:514
successfully appended: 61012f831ef37d32d31be790
total appended:515
successfully appended: 614fc2103d143144c7958a79
total appended:516
successfully appended: 614db565d6971e44f8af2c74
total appended:517
successfully appended: 6134d56ad6971e44f8a73555
total appended:518
successfully appended: 63e3ab9696fc8b6d043f71c8
total appended:519
successfully appended: 624046758b136ae3b737541a
total appended:520
successfully appended: 637c009305760e31507b7ce5
total appended:521
successfully appended: 60a65ad00c819813651f5bcd
total appended:522
successfully appended: 6386e4789c7242cc85599047
total appended:523
successfully appended: 607cb2634a065e6ea4b62d29
total appended:524
successfully appended: 624399158b136ae3b738a405
total appended:525
successfully appended: 623194f8ccd8656984d17f4c
total appended

successfully appended: 5f4efc03fdcc602e78a69b83
total appended:629
successfully appended: 5ecfae50e059476c0199b9f9
total appended:630
successfully appended: 61f14a1e8753a24060b7ad09
total appended:631
successfully appended: 5ef7e1adfd49e459caadefa3
total appended:632
successfully appended: 5f1b1a99f4b5fa115c65f293
total appended:633
successfully appended: 643f990a3ddca5a2e5b04736
total appended:634
successfully appended: 6030455395d4a33b1b6831bc
total appended:635
successfully appended: 6082aca94a065e6ea4b7f0ef
total appended:636
successfully appended: 5ed4b847ad2b9b6c11313c1f
total appended:637
successfully appended: 600ee903413441e94a589ff9
total appended:638
successfully appended: 5e7bcd52bc5a5a5832d059d9
total appended:639
successfully appended: 5ebe05480cfca96246a9cb80
total appended:640
successfully appended: 5e9c01a183e2a23fbb0139ee
total appended:641
successfully appended: 5f0a5a44f7075251cd9831f9
total appended:642
successfully appended: 5f4910367dad192e851793af
total appended

In [188]:
df_sprint.head()

,seed,lines,level_lines,level_lines_needed,inputs,holds,score,zenlevel,zenprogress,level,combo,currentcombopower,topcombo,btb,topbtb,currentbtbchainpower,tspins,piecesplaced,kills,finalTime,gametype,replayid,time.start,time.zero,time.locked,time.prev,time.frameoffset,clears.singles,clears.doubles,clears.triples,clears.quads,clears.realtspins,clears.minitspins,clears.minitspinsingles,clears.tspinsingles,clears.minitspindoubles,clears.tspindoubles,clears.tspintriples,clears.tspinquads,clears.allclear,garbage.sent,garbage.received,garbage.attack,garbage.cleared,finesse.combo,finesse.faults,finesse.perfectpieces
0,1.506296e+09,40,0,1,408,23.0,12096,1,0,1,2,0.0,2,0,5,0.0,0,108,0,62650.000000,40l,64258f17fb49ed069e697a91,0,False,False,0,0,11,1,1,6,0,0,0,0,0,0,0,0,0,0,0,0,0,0,74,67
1,4.994133e+08,40,0,1,358,23.0,10802,1,0,1,1,0.0,6,0,1,0.0,0,102,0,79260.000000,40l,640c4cbaaa4882e7599cfa0b,0,False,False,0,0,8,6,4,2,0,0,0,0,0,0,0,0,0,0,0,0,0,8,41,80
2,9.320079e+08,40,0,1,353,3.0,9142,1,0,1,2,0.0,3,0,0,0.0,0,102,0,59181.666667,40l,63e8f77ff0cc508ed05d9475,0,False,False,0,0,18,8,2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,56,70
3,1.302818e+08,41,0,1,643,9.0,9111,1,0,1,1,0.0,3,0,0,0.0,0,107,0,104233.333333,40l,644a7a3844b3865c688c9fff,0,False,False,0,0,26,6,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,292,24
4,7.581925e+08,40,0,1,441,27.0,12574,1,0,1,1,0.0,11,0,1,NaN,2,104,0,103871.666667,40l,63115f26cc59143b0f706c37,0,False,False,0,0,20,9,0,0,0,0,2,0,0,0,0,0,0,0,0,0,0,0,101,58


In [196]:
df_sprint.describe()

,seed,lines,level_lines,level_lines_needed,inputs,holds,score,zenlevel,zenprogress,level,combo,currentcombopower,topcombo,btb,topbtb,currentbtbchainpower,tspins,piecesplaced,kills,finalTime,time.start,time.prev,time.frameoffset,clears.singles,clears.doubles,clears.triples,clears.quads,clears.realtspins,clears.minitspins,clears.minitspinsingles,clears.tspinsingles,clears.minitspindoubles,clears.tspindoubles,clears.tspintriples,clears.tspinquads,clears.allclear,garbage.sent,garbage.received,garbage.attack,garbage.cleared,finesse.combo,finesse.faults,finesse.perfectpieces
count,6.630000e+02,663.000000,663.0,663.0,663.000000,631.000000,663.000000,663.0,663.0,663.0,663.000000,598.0,663.000000,663.000000,663.000000,519.0,663.000000,663.000000,663.0,663.000000,663.0,663.0,663.0,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.000000,663.0,663.000000,663.000000,663.0,663.000000,663.0,663.0,663.0,663.0,663.000000,663.000000,663.000000
mean,1.110052e+09,40.180995,0.0,1.0,444.607843,18.930269,11877.194570,1.0,0.0,1.0,1.514329,0.0,3.334842,0.782805,3.274510,0.0,0.411765,105.165913,0.0,93382.694822,0.0,0.0,0.0,11.277526,3.965309,1.307692,4.098039,0.027149,0.025641,0.027149,0.072398,0.0,0.220211,0.039216,0.0,0.098039,0.0,0.0,0.0,0.0,3.723982,109.539970,59.630468
std,6.284178e+08,0.473274,0.0,0.0,105.161665,12.603535,4001.872556,0.0,0.0,0.0,1.086594,0.0,1.921285,2.494321,3.130601,0.0,1.716540,4.983621,0.0,50199.041753,0.0,0.0,0.0,8.473429,2.836481,1.213737,3.301704,0.171678,0.176249,0.203857,0.510501,0.0,1.225268,0.298487,0.0,0.577299,0.0,0.0,0.0,0.0,10.776533,85.704779,21.780852
min,5.145320e+05,40.000000,0.0,1.0,264.000000,0.000000,7643.000000,1.0,0.0,1.0,1.000000,0.0,1.000000,0.000000,0.000000,0.0,0.000000,100.000000,0.0,14525.000000,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.000000,2.000000
25%,5.740935e+08,40.000000,0.0,1.0,369.000000,9.000000,9550.000000,1.0,0.0,1.0,1.000000,0.0,2.000000,0.000000,1.000000,0.0,0.000000,102.000000,0.0,57292.500000,0.0,0.0,0.0,4.000000,2.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,49.000000,44.000000
50%,1.071861e+09,40.000000,0.0,1.0,425.000000,18.000000,10925.000000,1.0,0.0,1.0,1.000000,0.0,3.000000,0.000000,2.000000,0.0,0.000000,104.000000,0.0,84628.333333,0.0,0.0,0.0,10.000000,4.000000,1.000000,3.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,1.000000,92.000000,60.000000
75%,1.683334e+09,40.000000,0.0,1.0,505.000000,27.000000,13177.000000,1.0,0.0,1.0,2.000000,0.0,4.000000,0.000000,5.000000,0.0,0.000000,107.000000,0.0,117718.333333,0.0,0.0,0.0,17.000000,6.000000,2.000000,7.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,3.000000,151.000000,75.000000
max,2.143527e+09,43.000000,0.0,1.0,1061.000000,63.000000,42177.000000,1.0,0.0,1.0,14.000000,0.0,17.000000,20.000000,20.000000,0.0,20.000000,138.000000,0.0,331446.666667,0.0,0.0,0.0,38.000000,13.000000,6.000000,10.000000,2.000000,2.000000,3.000000,6.000000,0.0,20.000000,5.000000,0.0,8.000000,0.0,0.0,0.0,0.0,103.000000,713.000000,104.000000


**I drop the variables that are the same value for every run (0 Standard Deviation)**

In [193]:
df_sprint2 = df_sprint.drop(['level_lines','level_lines_needed','zenlevel',
             'zenprogress','level','currentcombopower','kills',
             'time.start','time.prev','time.frameoffset','gametype',
             'clears.minitspindoubles','clears.tspinquads','garbage.sent',
             'garbage.received','garbage.attack','garbage.cleared','currentbtbchainpower','time.zero','time.locked'],axis=1)

In [195]:
df_sprint2.head()

,seed,lines,inputs,holds,score,combo,topcombo,btb,topbtb,tspins,piecesplaced,finalTime,replayid,clears.singles,clears.doubles,clears.triples,clears.quads,clears.realtspins,clears.minitspins,clears.minitspinsingles,clears.tspinsingles,clears.tspindoubles,clears.tspintriples,clears.allclear,finesse.combo,finesse.faults,finesse.perfectpieces
0,1.506296e+09,40,408,23.0,12096,2,2,0,5,0,108,62650.000000,64258f17fb49ed069e697a91,11,1,1,6,0,0,0,0,0,0,0,0,74,67
1,4.994133e+08,40,358,23.0,10802,1,6,0,1,0,102,79260.000000,640c4cbaaa4882e7599cfa0b,8,6,4,2,0,0,0,0,0,0,0,8,41,80
2,9.320079e+08,40,353,3.0,9142,2,3,0,0,0,102,59181.666667,63e8f77ff0cc508ed05d9475,18,8,2,0,0,0,0,0,0,0,0,1,56,70
3,1.302818e+08,41,643,9.0,9111,1,3,0,0,0,107,104233.333333,644a7a3844b3865c688c9fff,26,6,1,0,0,0,0,0,0,0,0,0,292,24
4,7.581925e+08,40,441,27.0,12574,1,11,0,1,2,104,103871.666667,63115f26cc59143b0f706c37,20,9,0,0,0,0,2,0,0,0,0,0,101,58


**Drop  NA values**

In [197]:
nan_values = df_sprint2.drop(index=382).isna()

columns_with_nan = nan_values.any(axis=0)

print(columns_with_nan)

seed                       False
lines                      False
inputs                     False
holds                       True
score                      False
combo                      False
topcombo                   False
btb                        False
topbtb                     False
tspins                     False
piecesplaced               False
finalTime                  False
replayid                   False
clears.singles             False
clears.doubles             False
clears.triples             False
clears.quads               False
clears.realtspins          False
clears.minitspins          False
clears.minitspinsingles    False
clears.tspinsingles        False
clears.tspindoubles        False
clears.tspintriples        False
clears.allclear            False
finesse.combo              False
finesse.faults             False
finesse.perfectpieces      False
dtype: bool


**Holds can be manually checked in the runs**

In [200]:
df_sprint2.to_csv(f"sprint_data.csv",index=False)